In [ ]:
from  google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.utils import resample
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import warnings


In [ ]:
data= pd.read_csv('/content/drive/MyDrive/sentiment_data/training_1600000_processed_noemoticon.csv',encoding='windows-1252')

In [ ]:
data.head()

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
data.columns = ['Sentiment','id','date','query','user','text']
data.head()

,Sentiment,id,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
data = data[['Sentiment','text']]
data.head()

,Sentiment,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
data.Sentiment.value_counts()


0    799996
4    248576
Name: Sentiment, dtype: int64

In [ ]:
data['Sentiment'] = data['Sentiment'].replace({4:1})

data.Sentiment.value_counts()


0    799996
1    248576
Name: Sentiment, dtype: int64

In [ ]:
data_major = data[data['Sentiment']==0]
data_minor = data[data['Sentiment']==1]


,Sentiment,text
799996,1,I LOVE @Health4UandPets u guys r the best!!
799997,1,im meeting up with one of my besties tonight! ...
799998,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
799999,1,Being sick can be really cheap when it hurts t...
800000,1,@LovesBrooklyn2 he has that effect on everyone


In [ ]:
data_major_downsampled = resample(data_major, 
                                 replace=False,   
                                 n_samples=len(data_minor),    
                                 random_state=1234) 

In [ ]:
df = data_major_downsampled.append(data_minor)
df.head()

,Sentiment,text
74567,0,Wow slept for almost 12hours. Sleepy me!! Uni ...
668722,0,gets bored with an idea too easily ... like tw...
286706,0,To my girls - sorry i've been a homebody latel...
632911,0,BK once again for the weekend...If it wasnt fo...
356735,0,@DonnieWahlberg Now why didn't you do that las...


In [ ]:
## remove stopwords and punctuation marks
stuff_to_be_removed = list(stopwords.words('english'))+list(punctuation)
stemmer = LancasterStemmer()

corpus = data['text'].tolist()
print(len(corpus))
print(corpus[0])

1048572
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!


In [ ]:
%%time
final_corpus = []
final_corpus_joined = []
for i in data.index:

    text = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    #Convert to lowercase
    text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()

    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text 
            if not word in stuff_to_be_removed] 
    text1 = " ".join(text)
    final_corpus.append(text)
    final_corpus_joined.append(text1)

CPU times: user 1min 16s, sys: 1.08 s, total: 1min 17s
Wall time: 1min 18s


In [ ]:
data_cleaned = pd.DataFrame()
data_cleaned["text"] = final_corpus_joined
data_cleaned["Sentiment"] = data["Sentiment"].values

In [ ]:
data_cleaned.head()

,text,Sentiment
0,upset update facebook texting might cry result...,0
1,kenichan dived many time ball managed save res...,0
2,whole body feel itchy like fire,0
3,nationwideclass behaving mad see,0
4,kwesidei whole crew,0


In [ ]:
negative = data_cleaned[data_cleaned['Sentiment'] == 0]
positive = data_cleaned[data_cleaned['Sentiment'] == 1]

In [ ]:
from sklearn.model_selection import train_test_split
destination_folder = '/content/drive/MyDrive/sentiment_data_saving'

In [ ]:
negative_full_train, negative_test = train_test_split(negative, train_size = .10, random_state = 1)
positive_full_train, positive_test = train_test_split(positive, train_size = .10, random_state = 1)

In [ ]:
negative_train, negative_valid = train_test_split(negative_full_train, train_size = .80, random_state = 1)
positive_train, positive_valid = train_test_split(positive_full_train, train_size = .80, random_state = 1)

In [ ]:
data_train = pd.concat([negative_train, positive_train], ignore_index=True, sort=False)
data_valid = pd.concat([negative_valid, positive_valid], ignore_index=True, sort=False)
data_test = pd.concat([negative_test, positive_test], ignore_index=True, sort=False)

In [ ]:
data_train = data_train[data_train['text']!=""]
data_valid = data_valid[data_valid['text']!=""]
data_test = data_test[data_test['text']!=""]



In [ ]:
data_train[data_train['text']==""]

,text,Sentiment


In [ ]:
data_train.to_csv(destination_folder + '/train.csv', index=False)
data_valid.to_csv(destination_folder + '/valid.csv', index=False)
data_test.to_csv(destination_folder + '/test.csv', index=False)